In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

# Load JSON files
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# Paths to dataset files (update these if needed)
train_file = "/content/drive/MyDrive/Colab Notebooks/train_preprocessed.json"
val_file = "/content/drive/MyDrive/Colab Notebooks/validation_preprocessed.json"
test_file = "/content/drive/MyDrive/Colab Notebooks/test_preprocessed.json"

# Load datasets
train_data = load_json(train_file)
val_data = load_json(val_file)
test_data = load_json(test_file)

# Extract sentences and entity labels
def extract_data(data):
    sentences = []
    labels = []
    for entry in data:
        tokens = entry.get("tokens", [])
        entities = entry.get("ner_tags", [])
        if tokens and entities:
            sentences.append(tokens)
            labels.append(entities)
    return sentences, labels

# Process datasets
train_sentences, train_labels = extract_data(train_data)
val_sentences, val_labels = extract_data(val_data)
test_sentences, test_labels = extract_data(test_data)

# Display sample data
print("Sample Training Sentence:", train_sentences[0])
print("Sample Training Labels:", train_labels[0])
print(f"Total Training Samples: {len(train_sentences)}")
print(f"Total Validation Samples: {len(val_sentences)}")
print(f"Total Test Samples: {len(test_sentences)}")


Sample Training Sentence: ['robert', 'gottschalk', '1939', 'academy', 'award', 'winner', 'and', 'founder', 'of', 'panavision']
Sample Training Labels: ['B-OtherPER', 'I-OtherPER', 'O', 'B-VisualWork', 'I-VisualWork', 'O', 'O', 'O', 'O', 'B-ORG']
Total Training Samples: 16778
Total Validation Samples: 871
Total Test Samples: 249980


In [ ]:
import random

# Process datasets
train_sentences, train_labels = extract_data(train_data)
val_sentences, val_labels = extract_data(val_data)
test_sentences, test_labels = extract_data(test_data)

# Randomly select 1000 samples for training (low-resource setting)
data_pairs = list(zip(train_sentences, train_labels))
random.seed(42)  # Ensure reproducibility
random.shuffle(data_pairs)
train_samples = data_pairs[:500]  # Select 1000 random samples
train_sentences, train_labels = zip(*train_samples)  # Unzip back to sentences and labels

print(f"Total Training Samples (Randomly Selected): {len(train_sentences)}")
print(f"Total Validation Samples: {len(val_sentences)}")
print(f"Total Test Samples: {len(test_sentences)}")

Total Training Samples (Randomly Selected): 500
Total Validation Samples: 871
Total Test Samples: 249980


In [ ]:
pip install accelerate nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
import random
import json
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [ ]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load XLM-RoBERTa-large for tokenization & embeddings (as in the paper)
XLM_MODEL_NAME = "xlm-roberta-large"
xlm_tokenizer = AutoTokenizer.from_pretrained(XLM_MODEL_NAME)
xlm_model = AutoModel.from_pretrained(XLM_MODEL_NAME).to(device).half()  # Move to GPU and use float16

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Load GPT-J-6B for text generation
GPTJ_MODEL_NAME = "EleutherAI/gpt-j-6B"
gptj_tokenizer = AutoTokenizer.from_pretrained(GPTJ_MODEL_NAME)
gptj_model = AutoModelForCausalLM.from_pretrained(
    GPTJ_MODEL_NAME,
    torch_dtype=torch.float16,  # Use float16 for efficiency
    device_map="auto",
    low_cpu_mem_usage=True
)

tokenizer_config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/24.2G [00:00<?, ?B/s]

Some weights of the model checkpoint at EleutherAI/gpt-j-6B were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.20.attn.bi

In [ ]:
# Function to get XLM-R embeddings
def get_xlm_embeddings(sentence):
    inputs = xlm_tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = xlm_model(**inputs)
    return outputs.last_hidden_state  # Extract contextual embeddings

In [ ]:
# Select 200 random samples for LLM augmentation
num_samples_for_llm = 10
llm_samples = random.sample(train_samples, num_samples_for_llm)

In [ ]:
# Function to select few-shot examples dynamically
def get_few_shot_examples(data, num_examples=3):
    sampled_data = random.sample(data, min(num_examples, len(data)))
    formatted_examples = "\n".join(
        [f"Sentence: {' '.join(entry[0])}\nEntities: {entry[1]}" for entry in sampled_data]
    )
    return formatted_examples

In [ ]:
def generate_augmented_data(sentence, labels, few_shot_data, num_augmentations=3, use_incontext=False):
    """Generates multiple augmented versions of a sentence in a single call."""

    if use_incontext:
        prompt = f"""You are an AI trained to generate diverse Named Entity Recognition (NER) data.
Your task is to rewrite the given sentence while preserving entity structure but changing context.
Ensure that the new sentences are significantly different from the original in wording.

Here are examples:
{few_shot_data}

Now, generate {num_augmentations} variations for this:
Sentence: {" ".join(sentence)}
Entities: {labels}

Output each sentence on a new line, without additional comments.
"""
    else:
        prompt = f"""Generate {num_augmentations} variations of the following sentence while maintaining named entity structure.

Original Sentence: {" ".join(sentence)}
Entity Tags: {labels}

Output each sentence on a new line, without additional comments.
"""

    # Tokenize input
    input_ids = gptj_tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.to(device)
    # input_ids = inputs.input_ids.to(device)
    # attention_mask = inputs.attention_mask.to(device)

    # Generate multiple outputs at once
    with torch.no_grad():
        outputs = gptj_model.generate(
            input_ids,
            # attention_mask=attention_mask,
            max_new_tokens=64,  # Generate multiple sentences at once
            temperature=0.8,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            # pad_token_id=gptj_tokenizer.pad_token_id
        )

    generated_text = gptj_tokenizer.decode(outputs[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return generated_text.strip().split("\n")  # Return multiple sentences as a list


In [ ]:
# Function to filter generated data
def is_diverse(original, generated):
    """Check if the generated sentence is sufficiently different from the original."""
    original_tokens = set(word_tokenize(" ".join(original)))
    generated_tokens = set(word_tokenize(generated))
    overlap = len(original_tokens & generated_tokens) / len(original_tokens)
    return overlap < 0.5  # Accept if less than 50% overlap

In [ ]:
# Perform augmentation
augmented_data = []
num_augmentations = 3  # Number of augmentations per sentence

In [ ]:
for sentence, labels in llm_samples:  # Use only selected 200 samples
    few_shot_examples = get_few_shot_examples(train_samples)  # Use all 1000 for context

    # Generate multiple augmentations at once
    augmented_sentences_1 = generate_augmented_data(sentence, labels, few_shot_examples, num_augmentations, use_incontext=False)
    augmented_sentences_2 = generate_augmented_data(sentence, labels, few_shot_examples, num_augmentations, use_incontext=True)

    # Store results
    for aug_sentence in augmented_sentences_1:
        augmented_data.append({"tokens": aug_sentence.split(), "ner_tags": labels})

    for aug_sentence in augmented_sentences_2:
        augmented_data.append({"tokens": aug_sentence.split(), "ner_tags": labels})


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [ ]:
# Save augmented data
with open("/content/drive/MyDrive/Colab Notebooks/augmented_data_exogenous.json", "w", encoding="utf-8") as f:
    json.dump(augmented_data, f, indent=4)

print(f"Augmentation complete! {len(augmented_data)} new sentences generated.")

Augmentation complete! 141 new sentences generated.


In [ ]:
from datasets import Dataset

# Load augmented data (either filtered or unfiltered)
with open("/content/drive/MyDrive/Colab Notebooks/augmented_data_exogenous.json", "r", encoding="utf-8") as f:
    augmented_data = json.load(f)

# Merge original 1000 training samples + augmented data
final_train_data = list(train_samples) + [(sample["tokens"], sample["ner_tags"]) for sample in augmented_data]

# Convert to Hugging Face dataset format
def convert_to_dataset(data):
    return Dataset.from_dict({"tokens": [x[0] for x in data], "ner_tags": [x[1] for x in data]})

train_dataset = convert_to_dataset(final_train_data)


In [ ]:
# We will look into filtering techniques as below code was not up to the mark



# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # Function to get sentence embeddings using XLM-RoBERTa
# def get_sentence_embedding(sentence):
#     inputs = xlm_tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
#     with torch.no_grad():
#         outputs = xlm_model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Get mean-pooled embeddings

# # Function to check Named Entity Consistency
# def check_named_entity_consistency(original_tokens, generated_tokens, labels):
#     original_entities = [word for word, label in zip(original_tokens, labels) if label != "O"]
#     generated_entities = [word for word in generated_tokens if word in original_entities]

#     # Check if at least 80% of named entities are retained
#     if len(original_entities) > 0:
#         entity_retention_ratio = len(generated_entities) / len(original_entities)
#         return entity_retention_ratio >= 0.3
#     return False

# # Function to check Semantic Similarity using XLM-R embeddings
# def is_diverse_enough(original_sentence, generated_sentence, threshold=0.95):
#     orig_embed = get_sentence_embedding(" ".join(original_sentence))
#     gen_embed = get_sentence_embedding(" ".join(generated_sentence))
#     similarity = cosine_similarity(orig_embed, gen_embed)[0][0]  # Compute cosine similarity
#     return similarity < threshold  # Accept if similarity is below threshold

# # Apply filtering
# filtered_data = []
# for sample in augmented_data:
#     original_sentence = sample["tokens"]
#     generated_sentence = sample["tokens"]  # Generated sentence is stored in the same key
#     labels = sample["ner_tags"]

#     # Convert to token lists for consistency check
#     original_tokens = word_tokenize(" ".join(original_sentence))
#     generated_tokens = word_tokenize(" ".join(generated_sentence))

#     # Apply all filtering conditions
#     if (check_named_entity_consistency(original_tokens, generated_tokens, labels) and
#         is_diverse_enough(original_tokens, generated_tokens)):
#         filtered_data.append(sample)

# # Save filtered data
# with open("/content/drive/MyDrive/Colab Notebooks/filtered_augmented_data.json", "w", encoding="utf-8") as f:
#     json.dump(filtered_data, f, indent=4)

# print(f"Filtering complete! {len(filtered_data)} samples retained after quality checks.")


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
print(f"Validation Sentences: {len(val_sentences)}")
print(f"Validation Labels: {len(val_labels)}")


Validation Sentences: 871
Validation Labels: 871


In [ ]:
# Load label mappings from file
with open("/content/drive/MyDrive/Colab Notebooks/label_mappings.json", "r", encoding="utf-8") as f:
    label_mapping = json.load(f)

# Extract correct mappings
label2id = label_mapping["label_to_id"]  # Correctly extract label-to-ID mapping
id2label = {int(k): v for k, v in label_mapping["id_to_label"].items()}  # Convert keys to int

# Debugging check
print("Corrected label2id:", label2id)
print("Corrected id2label:", id2label)


Corrected label2id: {'B-AerospaceManufacturer': 0, 'B-AnatomicalStructure': 1, 'B-ArtWork': 2, 'B-Artist': 3, 'B-Athlete': 4, 'B-CarManufacturer': 5, 'B-Cleric': 6, 'B-Clothing': 7, 'B-Disease': 8, 'B-Drink': 9, 'B-Facility': 10, 'B-Food': 11, 'B-HumanSettlement': 12, 'B-MedicalProcedure': 13, 'B-Medication/Vaccine': 14, 'B-MusicalGRP': 15, 'B-MusicalWork': 16, 'B-ORG': 17, 'B-OtherLOC': 18, 'B-OtherPER': 19, 'B-OtherPROD': 20, 'B-Politician': 21, 'B-PrivateCorp': 22, 'B-PublicCorp': 23, 'B-Scientist': 24, 'B-Software': 25, 'B-SportsGRP': 26, 'B-SportsManager': 27, 'B-Station': 28, 'B-Symptom': 29, 'B-Vehicle': 30, 'B-VisualWork': 31, 'B-WrittenWork': 32, 'I-AerospaceManufacturer': 33, 'I-AnatomicalStructure': 34, 'I-ArtWork': 35, 'I-Artist': 36, 'I-Athlete': 37, 'I-CarManufacturer': 38, 'I-Cleric': 39, 'I-Clothing': 40, 'I-Disease': 41, 'I-Drink': 42, 'I-Facility': 43, 'I-Food': 44, 'I-HumanSettlement': 45, 'I-MedicalProcedure': 46, 'I-Medication/Vaccine': 47, 'I-MusicalGRP': 48, 'I-M

In [ ]:
import gc
import torch

# Run garbage collection
gc.collect()

# Free up all possible GPU memory
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


In [ ]:
!nvidia-smi


Wed Mar 26 20:46:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   74C    P0             32W /   70W |   14430MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
os._exit(00)  # Force restart kernel


In [ ]:
# # Delete GPT-J safely before loading XLM-RoBERTa
# del gptj_model
# import gc
# gc.collect()
# torch.cuda.empty_cache()

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#### training TRY

In [ ]:
# Import necessary modules
import json
import random
import torch
import numpy as np
import nltk
from datasets import Dataset
from transformers import AutoModelForTokenClassification, AutoTokenizer, TrainingArguments, Trainer
from sklearn.metrics import f1_score

In [ ]:
# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Load original training data
with open("/content/drive/MyDrive/Colab Notebooks/train_preprocessed.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Load augmented data
with open("/content/drive/MyDrive/Colab Notebooks/augmented_data_exogenous.json", "r", encoding="utf-8") as f:
    augmented_data = json.load(f)

# Debug: Check structure of both datasets
print("🔹 Sample from train_preprocessed.json:")
print(json.dumps(train_data[:2], indent=4))  # Print first 2 samples

print("\n🔹 Sample from augmented_data_exogenous.json:")
print(json.dumps(augmented_data[:2], indent=4))  # Print first 2 samples

# Extract sentences & labels function
def extract_data(data):
    sentences, labels = [], []
    for entry in data:
        tokens = entry.get("tokens", [])
        entities = entry.get("ner_tags", [])
        if tokens and entities:
            sentences.append(tokens)
            labels.append(entities)
    return sentences, labels

# Extract sentences and labels from training data
train_sentences, train_labels = extract_data(train_data)

# Debug: Print dataset lengths
print(f"\n🔹 Total Samples in train_preprocessed.json: {len(train_sentences)}")
print(f"🔹 Total Samples in augmented_data_exogenous.json: {len(augmented_data)}")

# Randomly select 500 samples from train_preprocessed.json
selected_indices = random.sample(range(len(train_sentences)), 500)
train_sentences = [train_sentences[i] for i in selected_indices]
train_labels = [train_labels[i] for i in selected_indices]

# Convert original train data into tuples
original_train_data = list(zip(train_sentences, train_labels))

# Debug: Print structure of the first selected training sample
print("\n🔹 Sample from selected 500 original training samples:")
print(original_train_data[:2])

# Debug: Check structure of augmented data before merging
print("\n🔹 Sample from augmented data before merging:")
print(augmented_data[:2])


🔹 Sample from train_preprocessed.json:
[
    {
        "tokens": [
            "robert",
            "gottschalk",
            "1939",
            "academy",
            "award",
            "winner",
            "and",
            "founder",
            "of",
            "panavision"
        ],
        "ner_tags": [
            "B-OtherPER",
            "I-OtherPER",
            "O",
            "B-VisualWork",
            "I-VisualWork",
            "O",
            "O",
            "O",
            "O",
            "B-ORG"
        ]
    },
    {
        "tokens": [
            "during",
            "the",
            "reign",
            "of",
            "the",
            "tongzhi",
            "emperor",
            "(",
            "r",
            ".",
            "1861",
            "\u2013",
            "1875",
            ")",
            ":"
        ],
        "ner_tags": [
            "O",
            "O",
            "O",
            "O",
            "O",
            "B-O

In [ ]:
# Function to clean augmented data intelligently
def clean_augmented_data(data):
    cleaned_data = []
    for entry in data:
        tokens = entry.get("tokens", [])
        ner_tags = entry.get("ner_tags", [])

        # Ignore completely empty samples
        if not tokens:
            continue

        # Fix misaligned ner_tags by truncating/padding to match tokens length
        if len(ner_tags) > len(tokens):
            ner_tags = ner_tags[:len(tokens)]  # Truncate if too long
        elif len(ner_tags) < len(tokens):
            ner_tags.extend(["O"] * (len(tokens) - len(ner_tags)))  # Pad with "O"

        cleaned_data.append((tokens, ner_tags))

    return cleaned_data

# Apply smart cleaning
cleaned_augmented_data = clean_augmented_data(augmented_data)

# Debugging: Check structure after cleaning
print(f"\n🔹 Total Valid Augmented Samples After Cleaning: {len(cleaned_augmented_data)}")
print(f"🔹 Sample from Cleaned Augmented Data: {cleaned_augmented_data[:2]}")

# Merge original 500 samples + cleaned augmented data
final_train_data = original_train_data + cleaned_augmented_data

# Debugging: Check final merged dataset
print(f"\n🔹 Final Training Dataset Size: {len(final_train_data)}")
print(f"🔹 Sample from Final Training Data: {final_train_data[:2]}")



🔹 Total Valid Augmented Samples After Cleaning: 104
🔹 Sample from Cleaned Augmented Data: [(['A:'], ['O']), (['This', 'is', 'pretty', 'easy', 'to', 'do', 'with', 'a', 'regex', 'replacement:'], ['O', 'O', 'B-Athlete', 'I-Athlete', 'O', 'O', 'O', 'O', 'O', 'O'])]

🔹 Final Training Dataset Size: 604
🔹 Sample from Final Training Data: [(['mario', 'andretti', 'and', 'bobby', 'rahal', 'battled', 'for', 'the', 'lead', 'early', 'in', 'the', 'first', 'half', '.'], ['B-Athlete', 'I-Athlete', 'O', 'B-Athlete', 'I-Athlete', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['modu', 'chanyu', 'dies', 'and', 'is', 'succeeded', 'by', 'his', 'son', 'laoshang'], ['B-Politician', 'I-Politician', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OtherPER'])]


In [ ]:
# Load validation data
with open("/content/drive/MyDrive/Colab Notebooks/validation_preprocessed.json", "r", encoding="utf-8") as f:
    val_data = json.load(f)

# Load test data
with open("/content/drive/MyDrive/Colab Notebooks/test_preprocessed.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Debug: Check structure of validation & test data
print("🔹 Sample from validation_preprocessed.json:")
print(json.dumps(val_data[:2], indent=4))  # Print first 2 samples

print("\n🔹 Sample from test_preprocessed.json:")
print(json.dumps(test_data[:2], indent=4))  # Print first 2 samples

# Extract sentences & labels function
def extract_data(data):
    sentences, labels = [], []
    for entry in data:
        tokens = entry.get("tokens", [])
        entities = entry.get("ner_tags", [])
        if tokens and entities and len(tokens) == len(entities):
            sentences.append(tokens)
            labels.append(entities)
    return sentences, labels

# Extract validation data
val_sentences, val_labels = extract_data(val_data)

# Extract test data
test_sentences, test_labels = extract_data(test_data)

# Select first 30% of test data
test_size = int(0.3 * len(test_sentences))
test_sentences = test_sentences[:test_size]
test_labels = test_labels[:test_size]

# Debug: Print dataset sizes
print(f"\n🔹 Total Validation Samples: {len(val_sentences)}")
print(f"🔹 Total Test Samples (30% Selected): {len(test_sentences)}")

# Debug: Print structure of extracted validation & test samples
print("\n🔹 Sample from Processed Validation Data:")
print(val_sentences[:2], val_labels[:2])

print("\n🔹 Sample from Processed Test Data:")
print(test_sentences[:2], test_labels[:2])


🔹 Sample from validation_preprocessed.json:
[
    {
        "tokens": [
            "eli",
            "lilly",
            "founder",
            "president",
            "of",
            "pharmaceutical",
            "company",
            "eli",
            "lilly",
            "and",
            "company"
        ],
        "ner_tags": [
            "B-OtherPER",
            "I-OtherPER",
            "O",
            "O",
            "O",
            "O",
            "O",
            "B-PublicCorp",
            "I-PublicCorp",
            "I-PublicCorp",
            "I-PublicCorp"
        ]
    },
    {
        "tokens": [
            "christoph",
            "haberland",
            "designed",
            "a",
            "new",
            "marble",
            "pulpit",
            "for",
            "the",
            "church",
            "which",
            "was",
            "built",
            "in",
            "italy",
            "in",
            "1793",
            

In [ ]:
# Fix test data selection (limit to 30% correctly)
correct_test_size = int(0.3 * len(test_data))
test_sentences = test_sentences[:correct_test_size]
test_labels = test_labels[:correct_test_size]

# Debug: Check corrected test dataset size
print(f"\n🔹 Corrected Total Test Samples (30% Selected): {len(test_sentences)}")

# Extract all unique labels from train, validation, and test sets
all_labels = set(label for labels in train_labels for label in labels) | \
             set(label for labels in val_labels for label in labels) | \
             set(label for labels in test_labels for label in labels)

# Create label mappings
label2id = {label: idx for idx, label in enumerate(sorted(all_labels))}
id2label = {idx: label for label, idx in label2id.items()}

# Debug: Print label mappings
print("\n🔹 Label to ID Mapping:", label2id)
print("\n🔹 ID to Label Mapping:", id2label)

# Convert labels from text to corresponding integers
final_train_data = [(tokens, [label2id[label] for label in labels]) for tokens, labels in final_train_data]
val_data_pairs = [(tokens, [label2id[label] for label in labels]) for tokens, labels in zip(val_sentences, val_labels)]
test_data_pairs = [(tokens, [label2id[label] for label in labels]) for tokens, labels in zip(test_sentences, test_labels)]

# Debug: Check structure after conversion
print("\n🔹 Sample Converted Train Data:", final_train_data[:2])
print("\n🔹 Sample Converted Validation Data:", val_data_pairs[:2])
print("\n🔹 Sample Converted Test Data:", test_data_pairs[:2])



🔹 Corrected Total Test Samples (30% Selected): 74994

🔹 Label to ID Mapping: {'B-AerospaceManufacturer': 0, 'B-AnatomicalStructure': 1, 'B-ArtWork': 2, 'B-Artist': 3, 'B-Athlete': 4, 'B-CarManufacturer': 5, 'B-Cleric': 6, 'B-Clothing': 7, 'B-Disease': 8, 'B-Drink': 9, 'B-Facility': 10, 'B-Food': 11, 'B-HumanSettlement': 12, 'B-MedicalProcedure': 13, 'B-Medication/Vaccine': 14, 'B-MusicalGRP': 15, 'B-MusicalWork': 16, 'B-ORG': 17, 'B-OtherLOC': 18, 'B-OtherPER': 19, 'B-OtherPROD': 20, 'B-Politician': 21, 'B-PrivateCorp': 22, 'B-PublicCorp': 23, 'B-Scientist': 24, 'B-Software': 25, 'B-SportsGRP': 26, 'B-SportsManager': 27, 'B-Station': 28, 'B-Symptom': 29, 'B-Vehicle': 30, 'B-VisualWork': 31, 'B-WrittenWork': 32, 'I-AerospaceManufacturer': 33, 'I-AnatomicalStructure': 34, 'I-ArtWork': 35, 'I-Artist': 36, 'I-Athlete': 37, 'I-CarManufacturer': 38, 'I-Cleric': 39, 'I-Clothing': 40, 'I-Disease': 41, 'I-Drink': 42, 'I-Facility': 43, 'I-Food': 44, 'I-HumanSettlement': 45, 'I-MedicalProcedure'

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

# Load tokenizer
XLM_MODEL_NAME = "xlm-roberta-large"
xlm_tokenizer = AutoTokenizer.from_pretrained(XLM_MODEL_NAME)

# Function to tokenize data & align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlm_tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map subwords to original tokens

        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)  # Ignore special tokens & padding
            elif word_id >= len(label):
                label_ids.append(-100)  # Prevent out-of-bounds indexing
            else:
                label_ids.append(label[word_id])  # Assign correct label ID

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Convert data to Hugging Face dataset format
def convert_to_dataset(data):
    return Dataset.from_dict({"tokens": [x[0] for x in data], "ner_tags": [x[1] for x in data]})

# Convert datasets
train_dataset = convert_to_dataset(final_train_data)
val_dataset = convert_to_dataset(val_data_pairs)
test_dataset = convert_to_dataset(test_data_pairs)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
val_dataset = val_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

# Debug: Print dataset samples
print("\n🔹 Sample Tokenized Training Data:", train_dataset[0])
print("\n🔹 Sample Tokenized Validation Data:", val_dataset[0])
print("\n🔹 Sample Tokenized Test Data:", test_dataset[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/604 [00:00<?, ? examples/s]

Map:   0%|          | 0/871 [00:00<?, ? examples/s]

Map:   0%|          | 0/74994 [00:00<?, ? examples/s]


🔹 Sample Tokenized Training Data: {'tokens': ['mario', 'andretti', 'and', 'bobby', 'rahal', 'battled', 'for', 'the', 'lead', 'early', 'in', 'the', 'first', 'half', '.'], 'ner_tags': [4, 37, 66, 4, 37, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66], 'input_ids': [0, 5786, 31, 2528, 3771, 136, 337, 60060, 8980, 141, 133868, 71, 100, 70, 37105, 39395, 23, 70, 5117, 23552, 6, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
XLM_MODEL_NAME = "xlm-roberta-large"
xlm_tokenizer = AutoTokenizer.from_pretrained(XLM_MODEL_NAME)

# Load model with correct number of labels
model = AutoModelForTokenClassification.from_pretrained(
    XLM_MODEL_NAME,
    num_labels=len(label2id),  # Use the correct label count
    id2label=id2label,
    label2id=label2id
).to(device).half()  # Use float16 for memory efficiency


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load model with correct number of labels
model = AutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-large",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./ner_model",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="no",  # Disable checkpointing for faster training
    learning_rate=1e-5,
    per_device_train_batch_size=4,  # Reduce batch size if out-of-memory (OOM) occurs
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=xlm_tokenizer,
)

# Start training
trainer.train()


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-f82034fcaf71>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://

Epoch,Training Loss,Validation Loss
1,1.533300,1.483287
2,1.283100,1.267158
3,1.134800,1.128706
4,0.972700,1.009160
5,0.893700,0.958625


TrainOutput(global_step=755, training_loss=1.2563540408153409, metrics={'train_runtime': 479.5851, 'train_samples_per_second': 6.297, 'train_steps_per_second': 1.574, 'total_flos': 701328306416640.0, 'train_loss': 1.2563540408153409, 'epoch': 5.0})

In [ ]:
# Select only 5% of test data
test_sample_size = int(0.05 * len(test_dataset))
test_dataset_small = test_dataset.select(range(test_sample_size))

# Debug: Print new test set size
print(f"\n🔹 Using only {len(test_dataset_small)} samples for evaluation (5% of test data).")



🔹 Using only 3749 samples for evaluation (5% of test data).


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Function to convert predictions into readable labels
def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    true_labels = [[id2label[label] for label in label_row if label != -100] for label_row in label_ids]
    pred_labels = [[id2label[pred] for pred, label in zip(pred_row, label_row) if label != -100]
                   for pred_row, label_row in zip(preds, label_ids)]
    return true_labels, pred_labels

# Run model on reduced test set
test_results = trainer.predict(test_dataset_small)

# Convert predictions to labels
true_labels, pred_labels = align_predictions(test_results.predictions, test_results.label_ids)

# Compute F1 score
report = classification_report(sum(true_labels, []), sum(pred_labels, []), digits=4)
print("\n🔹 Final Test Set Performance (F1 Score & Classification Report):\n")
print(report)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🔹 Final Test Set Performance (F1 Score & Classification Report):

                       precision    recall  f1-score   support

B-AnatomicalStructure     0.0000    0.0000    0.0000         3
            B-ArtWork     0.0000    0.0000    0.0000         6
             B-Artist     0.0755    0.8308    0.1385       786
            B-Athlete     0.0000    0.0000    0.0000       203
    B-CarManufacturer     0.0000    0.0000    0.0000         3
             B-Cleric     0.0000    0.0000    0.0000        45
            B-Disease     0.0000    0.0000    0.0000        10
           B-Facility     0.3000    0.0938    0.1429       128
               B-Food     0.0000    0.0000    0.0000         5
    B-HumanSettlement     0.3255    0.8217    0.4663       387
   B-MedicalProcedure     0.0000    0.0000    0.0000         2
 B-Medication/Vaccine     0.0000    0.0000    0.0000         4
         B-MusicalGRP     0.0000    0.0000    0.0000        32
        B-MusicalWork     0.0000    0.0000    0.00

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
